In [30]:
from utils.data import MyDataset, MySplitDataset

data_path = 'data/amazon-clothing'
words = 15
vocab_size = 20_000

data = MyDataset(data_path, words, vocab_size) # tarda entre 5 i 10 segons normalment. A vegades només 3 seg en re-execucions

In [31]:
import torch
from torch.utils.data import DataLoader, Subset

index_dir = '1'
batch_size = 128

mysplitdata = MySplitDataset(data_path, len(data), index_dir, True)

def collate_fn(batch):
    return [torch.tensor(x) for x in zip(*batch)]

train_data = Subset(data, mysplitdata.train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [32]:
len(train_data), len(train_loader.dataset)

(143378, 143378)

In [38]:
for batch in train_loader:
    break

In [40]:
[b.shape for b in batch] # el bo et dona les coses sempre així, tot perfecte amb la 1a dimensió el batch_size

[torch.Size([128]),
 torch.Size([128]),
 torch.Size([128]),
 torch.Size([128, 17]),
 torch.Size([128])]

In [33]:
 # Si no li dic la collate_fn, l'índex 3 és problemàtic (llista)
train_loader_without_collate = DataLoader(train_data, batch_size=32, shuffle=True)
for batch in train_loader_without_collate:
    print(len(batch))
    print([type(b) for b in batch])
    break

5
[<class 'torch.Tensor'>, <class 'torch.Tensor'>, <class 'torch.Tensor'>, <class 'list'>, <class 'torch.Tensor'>]


In [34]:
first_batch = next(iter(train_loader))

In [35]:
[(elem.shape, elem.dtype) for elem in first_batch]

[(torch.Size([128]), torch.int64),
 (torch.Size([128]), torch.int64),
 (torch.Size([128]), torch.float32),
 (torch.Size([128, 17]), torch.int64),
 (torch.Size([128]), torch.int64)]

In [36]:
user, item, rating, text, feature = first_batch

In [37]:
import pandas as pd

df = pd.DataFrame(data.untransform_batch(first_batch))

df

,user,item,rating,text,feature
0,A32E970U89EAUX,B009S3HT1I,4.0,so the dress is really pretty so i ca n't faul...,dress
1,A3FCK65KL97H9Z,B007WAEB10,5.0,the length is a good length and perfect with l...,length
2,A2MYCLMG0D3YR7,B004HHP00C,5.0,but the replacements fit perfectly,fit
3,A2OF4UHZ612JB3,B0085971YQ,4.0,it 's a very cute dress,dress
4,A1LT1URVMNXE6F,B00AUC4C3E,5.0,these shoes are soooo comfortable,sooo
...,...,...,...,...,...
123,A2N64UT67J8HFM,B003LXTWIO,5.0,and they are made with very strong fabric,fabric
124,A05613441GM5VUIRSCIK1,B001ARZP02,5.0,but loo and behold the boots fit him perfectly,boots
125,A10BXGWEGMAHSK,B00JB2IZ7I,4.0,shows everything so i 'd have to be in much be...,shape
126,ARWCQUR3TQZ3N,B004R1EMVQ,5.0,a deep v neck that highlights ur jewelery and ...,neck


In [26]:
data_no_feat = MyDataset(data_path, words, vocab_size, feature_col=None)

mysplitdata_no_feat = MySplitDataset(data_path, len(data_no_feat), index_dir, True)

train_data_no_feat = Subset(data_no_feat, mysplitdata_no_feat.train)
train_loader_no_feat = DataLoader(train_data_no_feat, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [27]:
first_batch_no_feat = next(iter(train_loader_no_feat))

In [28]:
df_no_feat = pd.DataFrame(data_no_feat.untransform_batch(first_batch_no_feat))

df_no_feat

,user,item,rating,text
0,A2K9EISKYABFQR,B009YRM6MK,5.0,i went up one size to be super comfortable
1,A2IAHYQEOVCNYE,B004IYZKWW,5.0,these shoes are extremely comfortable and help...
2,A3QMBPOM0FSBCC,B0085IFDEW,4.0,still a good shirt for a chilly day
3,A2XPT8ZZH17XGW,B002R0FAO2,4.0,very cute but runs small
4,A1HG82U7GY9531,B003UYTEO6,3.0,these jeans fit true to size
...,...,...,...,...
123,AQR1LRU8TLJ5H,B0064X6JGQ,5.0,dress fits perfectly & is very flattering for ...
124,AFEC3BV85IXS4,B008J4EZE6,5.0,they fit close to my feet which are wide
125,A3V26E528CWT3V,B0057017RS,3.0,easy wear
126,A1N5FL7SNKG1JV,B006QO3T7U,3.0,i was able to return it right away and get ano...
